# Decision making in high uncertainty environments : a behavioural analog to BCI Training

This notebook runs simulation for our behavioural experiment.

In [1]:
import actynf

print(actynf.__version__)


# ENVIRONMENTAL CONSTANTS :
GRID_SIZE = (7,7)
START_COORD = [[5,1],[5,2],[4,1]]
END_COORD = [0,6]

T = 10
Th = 2
action_cap = 3
state_cap = 3
structure_hypothesis = actynf.LINEAR

0.1.39


In [ ]:
from actynf import layer,link,layer_network
from .models import behavioural_process,naive_model,basic_latent_model,grid_latent_model

def agent_model_layer(model_parameters,N_fb_ticks,modeltype="naive"):
    A,B,C,D,E,U = behavioural_process(GRID_SIZE,START_COORD,END_COORD,N_fb_ticks)
    
    if modeltype=="naive":
        model = naive_model
    elif modeltype=="1d":
        model = basic_latent_model
    elif modeltype=="2d":
        model = grid_latent_model
    
    process = layer("environment","process",A,B,C,D,E,U,)
    
    for action_model in ["angle","position","distance"]:
        a,b,c,d,e,u = model(model_parameters,action_model)
        this_layer = layer("subject_"+modeltype+"_"+action_model,"model",a,b,c,d,e,u,T,Th)
        
        # layer additionnal options : 
        this_layer.hyperparams.alpha = 32
        this_layer.hyperparams.cap_action_explo = action_cap
        this_layer.hyperparams.cap_state_explo = state_cap

        this_layer.learn_options.eta = 10
        this_layer.learn_options.learn_a = False
        this_layer.learn_options.learn_b = True
        this_layer.learn_options.learn_d = True
        this_layer.learn_options.assume_state_space_structure = structure_hypothesis
        this_layer.learn_options.generalize_fadeout_function_temperature = generalize_temperature
        
        